TJCM

In [1]:
# Needs Uni enviornment

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [3]:
journal = 'tjcm20'

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

# searchTerm1 = ['a']
# searchTerm2 = ['the']

tc = 0
noResultsList = []

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

baseUrl = 'https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22'
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        query = baseUrl + item1 +"%22+AND+%22"+ item2+"%22&Ppub=&startPage=&SeriesKey=" + journal
        print(query)
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        tc = tc + number
                
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
#         print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize = 2

        if number == 0:
            noResultsList.append((item1 +","+item2))
            print("No results found")
        else:
            


            print("Extracting all Links")

            for currentPage in range(0,pageSize):
                print("In page: ",currentPage)

                if currentPage != 0:
                    newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                    cnt = cnt + 1
                else:
                    newQuery = query


                print("Searching for query: ",newQuery)
                response = requests.get(newQuery)
                soup = BeautifulSoup(response.text, 'html.parser')
                mydivs = soup.findAll("a", {'class':['ref nowrap']})

                for d in mydivs:
                    url = 'https://www.tandfonline.com' + d.get('href')
                    allLinks.append(url)
    #             print(allLinks)

            print("Extracting data")

            for link in allLinks:
                df.loc[row,'searchTerm1'] = item1
                df.loc[row,'searchTerm2'] = item2
                df.loc[row,'url'] = link
                html_content = requests.get(link).text
                soup = BeautifulSoup(html_content, "lxml")

                for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
    #                 print("Title: ",i)
                    df.loc[row,'Title'] = i

                temp = soup.findAll('h2',{'class':['section-heading-2']} )
                print("Number of sections: ",len(temp))            

                count = 0
                for i in range(len(temp)):

                    if temp[i].text == temp[i].find_next('p').text:

                        extract = "section" + str(count)
                        extractData = temp[i].find_next('p').find_next('p').text
                        df.loc[row,extract] = temp[i].text
                        sectionData = "sectionData" + str(count)                    
                        df.loc[row,sectionData] = extractData
                        count = count + 1
                    else:
                        extract = "section" + str(count)
                        extractData = temp[i].find_next('p').text
                        df.loc[row,extract] = temp[i].text
                        sectionData = "sectionData" + str(count)
                        df.loc[row,sectionData] = extractData
                        count = count + 1


                row = row + 1
                print("row: ",row)


['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tjcm20
Extracting all Links
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tjcm20
Extracting data
Number of sections:  10
row:  1
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonlin

Extracting all Links
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+Information+Modelling%22+AND+%22Energy%22&Ppub=&startPage=&SeriesKey=tjcm20
Extracting data
Number of sections:  0
row:  27
Number of sections:  10
row:  28
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+Information+Modelling%22+AND+%22Water%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+Information+Modelling%22+AND+%22Waste%22&Ppub=&startPage=&SeriesKey=tjcm20
Extracting all Links
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+Information+Modelling%22+AND+%22Waste%22&Ppub=&startPage=&SeriesKey=tjcm20
Extracting data
Number of sections:  8
row:  29
Number of sections:  5
row:  30
Number of sections:  6
row:  31
Number of sections:  9
row:  32
Number of sections:  7
row:  33
Num

No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Environmentally+friendly+design%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Sustainability%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Sustainable+development%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Energy%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Water%22&Ppub=&startPage=&SeriesKey=tjcm20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+construction%22+AND+%22Waste%22&Ppub=&startPage=&Ser

In [6]:
noResultsList

['BIM,Green+Buildings',
 'BIM,Water',
 'BIM,Thermal+comfort',
 'BIM,Electricity',
 'BIM,Solar',
 'BIM,Wind+power',
 'BIM,Green+infrastructure',
 'Building+Information+Modelling,Green+Buildings',
 'Building+Information+Modelling,Water',
 'Building+Information+Modelling,Thermal+comfort',
 'Building+Information+Modelling,Electricity',
 'Building+Information+Modelling,Solar',
 'Building+Information+Modelling,Wind+power',
 'Building+Information+Modelling,Green+infrastructure',
 'Building+information+model,Green+Building',
 'Building+information+model,Green+Buildings',
 'Building+information+model,Sustainable+building',
 'Building+information+model,Sustainable+buildings',
 'Building+information+model,Environmentally+friendly+design',
 'Building+information+model,Sustainability',
 'Building+information+model,Sustainable+development',
 'Building+information+model,Energy',
 'Building+information+model,Water',
 'Building+information+model,Waste',
 'Building+information+model,Thermal+comfort',
 '

In [7]:
df['journalName'] = journal

In [8]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Evaluation of 4D BIM tools applicability in co...,Abstract,Worldwide the construction industry presents a...,Introduction,Worldwide the construction industry is well kn...,Literature review,The term Building Information Modelling (BIM) ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
1,BIM,Sustainable+building,https://www.tandfonline.com/doi/full/10.1080/1...,Strategic analysis of barriers and solutions t...,Abstract,The construction industry is growing and due t...,Introduction,The construction industry has significant impa...,Literature review,The definition of sustainable development was ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
2,BIM,Sustainable+building,https://www.tandfonline.com/doi/full/10.1080/1...,BIM-integrated TOPSIS-Fuzzy framework to optim...,AbstractFormulae display:?Mathematical formula...,Abstract,Introduction,Since sustainable design for buildings turns t...,Literature review,Lu et al. (2007) presented three standard appr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
3,BIM,Sustainable+buildings,https://www.tandfonline.com/doi/full/10.1080/1...,Strategic analysis of barriers and solutions t...,Abstract,The construction industry is growing and due t...,Introduction,The construction industry has significant impa...,Literature review,The definition of sustainable development was ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
4,BIM,Environmentally+friendly+design,https://www.tandfonline.com/doi/abs/10.1080/15...,Using Building Information Modeling to Assess ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
5,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,Applying BIM to achieve sustainability through...,Abstract,The research addresses the issue that there is...,Introduction,Many architects have believed that using moder...,Research problem,The overall usage and operation potentials of ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
6,BIM,Sustainability,https://www.tandfonline.com/doi/abs/10.1080/15...,Using Building Information Modeling to Assess ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
7,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/1...,BIM-integrated TOPSIS-Fuzzy framework to optim...,AbstractFormulae display:?Mathematical formula...,Abstract,Introduction,Since sustainable design for buildings turns t...,Literature review,Lu et al. (2007) presented three standard appr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
8,BIM,Sustainable+development,https://www.tandfonline.com/doi/full/10.1080/1...,Proposing building information modeling-based ...,Abstract,Waste generated throughout the construction an...,Introduction,Construction industry is an excessive exploite...,Methodology,An extant literature review was undertaken and...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20
9,BIM,Energy,https://www.tandfonline.com/doi/abs/10.1080/15...,Using Building Information Modeling to Assess ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tjcm20


In [9]:
jName = 'kgRawData/KG-' + journal +'.csv'
df.to_csv(jName)

In [1]:
journalName = 'tjcm20'

import pandas as pd
df = pd.read_csv('kgRawData/KG-'+ journalName +'.csv')
df = df.where(pd.notnull(df), None)

import re
absRows = []
rowsFound = []
for i in range(len(df['url'])):
    text = df['url'][i]
    pattern = '/abs/'
    if re.search(pattern, text):
        absRows.append(i)
    else:
        rowsFound.append(i)
        
rowsFoundDf = pd.DataFrame()
rowsFoundDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

rowsFoundDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
rowsFoundDf = rowsFoundDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    rowsFoundDf[section] = ''
    rowsFoundDf[sectionData] = ''

    
absRowsDf = pd.DataFrame()
absRowsDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

absRowsDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
absRowsDf = absRowsDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    absRowsDf[section] = ''
    absRowsDf[sectionData] = ''
    
    
for entry in range(len(rowsFound)):
    rowsFoundDf.loc[entry] = df.loc[rowsFound[entry]]

for entry in range(len(absRows)):
    absRowsDf.loc[entry] = df.loc[absRows[entry]]

rowsFoundDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-full.csv')
absRowsDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-abs.csv')

print(len(absRowsDf))
print(len(rowsFoundDf))

6
29


In [3]:
absRowsDf['url'][0]

'https://www.tandfonline.com/doi/abs/10.1080/15623599.2012.10773184'